In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from lightgbm import LGBMClassifier

# Read the CSV file into a DataFrame
tweet_data_int = pd.read_csv('train_nick.csv')

# Convert 'tweet' column to string data type and remove non-string entries
tweet_data_int['tweet'] = tweet_data_int['tweet'].astype(str)
tweet_data_int = tweet_data_int[tweet_data_int['tweet'].apply(lambda x: isinstance(x, str))]

# Split the data into features (X) and target (y)
X = tweet_data_int['tweet']
y = tweet_data_int['sentiment']

# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the validation and testing data
X_val_vectorized = vectorizer.transform(X_val)
X_test_vectorized = vectorizer.transform(X_test)

# Define the parameter grid for grid search
param_grid = {
    'num_leaves': [20, 30],
    'max_depth': [10, 20],
    'n_estimators': [100, 200]
}

# Create the LightGBM classifier
lgbm = LGBMClassifier()

# Perform grid search to find the best parameters
grid_search = GridSearchCV(lgbm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_vectorized, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Get predictions on the test set
y_pred = best_model.predict(X_test_vectorized)

# Calculate accuracy for the best model
accuracy = accuracy_score(y_test, y_pred)

# Compare accuracy before and after tuning
y_pred_initial = best_model.predict(X_test_vectorized)
accuracy_initial = accuracy_score(y_test, y_pred_initial)

# Print accuracy and best parameters
print("Accuracy (Best Model):", accuracy)
print("Best Parameters:", best_params)
print("Accuracy (Initial Model):", accuracy_initial)

# Print classification report
print(classification_report(y_test, y_pred))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.128078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 105796
[LightGBM] [Info] Number of data points in the train set: 35517, number of used features: 2406
[LightGBM] [Info] Start training from score -1.770615
[LightGBM] [Info] Start training from score -1.193433
[LightGBM] [Info] Start training from score -1.404966
[LightGBM] [Info] Start training from score -1.268627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.251986 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not en